## Sniffer
---
"Sniff out good and bad images in your dataset"

In [1]:
from pynput import keyboard
import panel as pn
import pandas as pd
from datetime import datetime
import holoviews as hv
import jupyter_bokeh

from Sniffer import sniffer
pn.extension()
pn.extension(loading_spinner='dots', loading_color='#00aa41', sizing_mode="stretch_width")

# Make an instance of the Sniffer Class   
sniffer_app = sniffer.SnifferClass()

sniffer_app.thumbnail_button.on_click(sniffer_app.thumbnail_button_clicked)
sniffer_app.image_button.on_click(sniffer_app.image_button_clicked)
sniffer_app.yes_button.on_click(sniffer_app.yes_button_clicked)
sniffer_app.no_button.on_click(sniffer_app.no_button_clicked)
sniffer_app.undo_button.on_click(sniffer_app.undo_button_clicked)

image_mode_selection=pn.Row(sniffer_app.thumbnail_button,sniffer_app.image_button)

def on_activate_yes():
    sniffer_app.yes_hotkey()

def on_activate_no():
    sniffer_app.no_hotkey()
    
def on_activate_undo():
    sniffer_app.undo_hotkey()    

listener = keyboard.GlobalHotKeys({
        'a': on_activate_yes,
        'd': on_activate_no,
        's': on_activate_undo})

listener.start()

component = pn.Row(
    pn.Column(sniffer_app.yes_button, sniffer_app.no_button, sniffer_app.undo_button, background='#b9d5f0',width=100,height=250,margin=(55,5)),
    pn.Column(sniffer_app.get_photo_index_text,sniffer_app.text,image_mode_selection,sniffer_app.get_progress_bar, sniffer_app.get_jpg_panel,background='#b9d5f0', height= 670, width=500,margin=(5,0),align='start',sizing_mode='fixed'),
    pn.Column(sniffer_app.radio_group,background='#b9d5f0', width=230,height=250,margin=(55,5)),
    width=900,
    height=700,
    background='#6eb4f5'
    )
component

BokehModel(combine_events=True, render_bundle={'docs_json': {'6ec68f00-5d91-4bb5-ac1c-4058630d1a4b': {'defs': …

Row(background='#6eb4f5', height=700, sizing_mode='fixed', width=900)
    [0] Column(background='#b9d5f0', height=250, margin=(55, 5), sizing_mode='fixed', width=100)
        [0] Button(button_type='success', disabled=True, margin=(10, 20), name='YES', width=50)
        [1] Button(button_type='danger', disabled=True, margin=(10, 20), name='NO', width=50)
        [2] Button(button_type='warning', disabled=True, margin=(10, 20), name='UNDO', width=35)
    [1] Column(background='#b9d5f0', height=670, margin=(5, 0), sizing_mode='fixed', width=500)
        [0] ParamMethod(method, sizing_mode='stretch_width')
        [1] StaticText(margin=(5, 120), value='Click THUMBNAIL t...)
        [2] Row(sizing_mode='stretch_width')
            [0] Button(button_type='success', margin=(10, 20), name='THUMBNAIL', width=50)
            [1] Button(button_type='success', margin=(10, 20), name='IMAGE', width=50)
        [3] ParamMethod(method, sizing_mode='stretch_width')
        [4] ParamMethod(method, sizing_mode='stretch_width')
    [2] Column(background='#b9d5f0', height=250, margin=(55, 5), sizing_mode='fixed', width=230)
        [0] RadioButtonGroup(margin=(40, 1), name='Radio Button Group', options=['CSV Mode', 'File Mode'], value='CSV Mode')

In [ ]:
# Creates a template that can be served with the command
# panel serve Sniffer.ipynb from the command line
template = pn.template.MaterialTemplate(
    title='',
    logo='assets/sniffer_big.jpg',
    header_background='#b9d5f0',
    main=[component],
)
template.servable();
template